In [ ]:
import os
import numpy as np
import h5py
import tifffile as tiff

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rcParams['text.latex.unicode'] = False

In [ ]:
fname = 'VJ_OFCVTA_7_260_D6'
root_dir = 'C:\\2pData\\Vijay data\\VJ_OFCVTA_7_D8_trained\\'
suite2p_dat_dir = root_dir + 'suite2p\\plane0\\'

fig_save_dir = os.path.join(root_dir, 'figs')
proj_dir = os.path.join(root_dir, '{}_output_images'.format(fname))

# projections path
std_proj_path = os.path.join(proj_dir, 'std_img.tif')

In [ ]:
std_img = tiff.imread(std_proj_path)

In [ ]:
ops = np.load(suite2p_dat_dir + "ops.npy", allow_pickle = True).item()
iscell = np.load(suite2p_dat_dir + "iscell.npy", allow_pickle = True)

F = np.load(suite2p_dat_dir + 'F.npy')
Fneu = np.load(suite2p_dat_dir + 'Fneu.npy')
stat = np.load(suite2p_dat_dir + 'stat.npy', allow_pickle = True)

print('stat dict keys:')
print(stat[0].keys())

In [ ]:
cell_ids = np.where( iscell[:,0] == 1 )[0]  # indices of detected cells across all ROIs from suite2p

In [ ]:
# define number of ROIs to visualize and make colormap
numROI_2Viz = 5
colors_roi = plt.cm.viridis(np.linspace(0,1,numROI_2Viz))

In [ ]:
# initialize templates for contour map
plot_these_roi = []
zero_template = np.zeros([ops['Ly'], ops['Lx']])

# loop through ROIs and add their spatial footprints to template
for idx,iROI in enumerate(cell_ids):
    zero_template[ stat[iROI]['ypix'],stat[iROI]['xpix'] ] = 1*(idx+1)
    
    plot_these_roi.append(iROI) # just an array of cell ROI numbers for subsequent analyses
    
    if idx == numROI_2Viz-1:
        break


In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10,10))
ax.imshow(std_img, cmap = 'gray')
ax.axis('off')
ax.contour(zero_template, colors = colors_roi);

plt.savefig(fig_save_dir + 'roi_contour_map.jpg')
plt.savefig(fig_save_dir + 'roi_contour_map.pdf')

# Plot Time-series of Selected ROIs

In [ ]:
# initialize variables for plotting time-series
fs = ops['fs']
num_samps = ops['nframes']
total_time = num_samps/fs 
tvec = np.linspace(0,total_time,num_samps)

In [ ]:
fig, ax = plt.subplots(numROI_2Viz, 1, figsize = (9,10))
for idx, iROI in enumerate(plot_these_roi):
    
    baseline = np.mean(F[iROI])
    
    to_plot = ( (F[iROI]-baseline)/baseline )*100
    
    ax[idx].plot(tvec, np.transpose( to_plot ), color = colors_roi[idx] );
    ax[idx].axis([0,500,-100,500])
    
    if idx == np.ceil(numROI_2Viz/2-1):
        ax[idx].set_ylabel('Fluorescence Level',fontsize = 20)
        
ax[idx].set_xlabel('Time [s]',fontsize = 20);
plt.savefig(fig_save_dir + 'roi_ts.jpg')
plt.savefig(fig_save_dir + 'roi_ts.pdf')